# DataFrame Sorting, Ranking, and Uniqueness

In this chapter, we cover methods that sort, rank, and find unique values on the entire DataFrame. All the methods presented in this chapter appeared in the same chapter for Series with the exception of `unique` which is, ironically, unique to just the Series.

## Sorting

pandas allows you to sort the rows of a DataFrame either by the values or by the index with the `sort_values` and `sort_index` methods.

### The `sort_values` method

The `sort_values` DataFrame method sorts the DataFrame by the values of one or more columns. Pass the `by` parameter a column name or list of column names to sort. By default, the sorting takes place in ascending manner (from least to greatest). The college dataset is used for the remainder of the examples in this chapter.

In [ ]:
import pandas as pd
college = pd.read_csv('../data/college.csv', index_col='instnm')
college.head(3)

Let's sort the rows by city.

In [ ]:
college.sort_values(by='city').head(3)

### Sort from greatest to least

Set the `ascending` parameter to `False` to sort in the opposite direction. Since the `by` parameter is first, it is usually omitted.

In [ ]:
college.sort_values('city', ascending=False).head(3)

### Simultaneously sort two or more columns

Sort by any number of columns by passing a list of their names to the `by` parameter. The sort begins with the first column. For instance, the following sorts all the colleges by state, and then within each state, sorts by undergraduate population.

In [ ]:
cols = ['stabbr', 'ugds']
state_ugds_sort = college.sort_values(cols)
state_ugds_sort[cols].head(3)

Let's select just the state of Oklahoma to verify that it too is sorted by undergraduate population.

In [ ]:
state_ugds_sort.query('stabbr == "OK"')[cols].head(3)

### Sort multiple columns in different directions

The `ascending` parameter may be passed a list of booleans corresponding to the list of column names in the `by` parameter. The following sorts by state from least to greatest, and then by undergraduate population from greatest to least.

In [ ]:
college[cols].sort_values(cols, ascending=[True, False]).head(3)

### Sort by the index

The DataFrame may be sorted by its index with the `sort_index` method.

In [ ]:
college.sort_index(ascending=False).head(3)

### Sort the columns

Interestingly, you can use the same `sort_index` method to sort the columns of the DataFrame. You must remember that pandas uses an Index object to contain the columns, which is the same object that contains the index. To sort the columns, set the `axis` parameter to 'columns' or 1. This is identical to how we changed the direction of the operation of the statistical methods in previous chapters. Perhaps this method would have been more appropriately named `sort_axis` instead since it sorts either axis.

In [ ]:
college.sort_index(axis=1).head(2)

## Ranking

The `rank` method ranks the values in each column, independently, the same way it does for a Series. Let's explore ranking on a subset of the columns from the movie dataset. To simplify matters, we will work with just the first 5 rows of data.

In [ ]:
movie = pd.read_csv('../data/movie.csv', index_col='title')
cols = ['year', 'director_name', 'duration', 'director_fb', 'actor1_fb', 
        'actor2_fb', 'actor3_fb', 'num_reviews', 'imdb_score']
movie_5 = movie[cols].head()
movie_5

Calling the `rank` method with the defaults ranks each value in ascending order beginning with 1. For instance, take a look at the `actor1_fb` column. The smallest value is the fifth value and is given rank 1 below. The `rank` method also ranks string columns.

In [ ]:
movie_5.rank()

Here, we rank from greatest to least using the minimum rank for ties which take place in the `director_fb` and `imdb_score` columns.

In [ ]:
movie_5.rank(ascending=False, method='min')

## Uniqueness

In this section we cover the `nunique` and `drop_duplicates` DataFrame methods which exist for Series. We'll use the City of Houston employee dataset for the next set of examples.

In [ ]:
emp = pd.read_csv('../data/employee.csv')
emp.head(3)

### The `nunique` method

The `nunique` method returns the number of unique values in each column as a Series. The old column names are now the new index values.

In [ ]:
emp.nunique()

To count any missing values as exactly one more unique, set the the `dropna` parameter to `False`.

In [ ]:
emp.nunique(dropna=False)

### The `drop_duplicates` method

The default call to the `drop_duplicates` method returns only unique rows of the DataFrame. It does not use the index value in its search for duplicates. If two or more rows are duplicated, the first row is kept. Let's see if there are any duplicate rows in the employee dataset.

In [ ]:
emp.shape

In [ ]:
emp.drop_duplicates().shape

Interestingly, there are some rows with the exact same information for all six columns.

### Drop duplicates based on a subset of columns

Instead of dropping rows where the entire row is duplicated, you can restrict the search for duplication to a subset of the columns. Pass the `subset` parameter a single column name or a list of column names. The following example returns a single row for each unique department.

In [ ]:
emp.drop_duplicates(subset='dept')

The following returns the first row for each unique combination of race and sex.

In [ ]:
emp.drop_duplicates(subset=['race', 'sex']).head(4)

## Finding the maximum/minimum of a group

In this section, we will explore a practical example of the `drop_duplicates` method. Let's say we are interested in finding the employee with the maximum salary per department. This results in a DataFrame with a single row for each department. Let's begin by sorting by department first and then salary, making sure salary is sorted from greatest to least.

In [ ]:
emp_sorted = emp.sort_values(['dept', 'salary'], ascending=[True, False])
emp_sorted.head(3)

The data is correctly sorted, but the information we want is not easily accessible. We desire a single row for each department. We can now turn to the `drop_duplicates` method and use the `subset` parameter to keep the first row of every department.

In [ ]:
emp_sorted.drop_duplicates(subset='dept')

We can rewrite our solution without assigning the result of `sort_values` to a variable by chaining the `drop_duplicates` method directly after it. Here, we find the employee with the lowest salary per department.

In [ ]:
(emp.sort_values(['dept', 'salary'])
    .drop_duplicates(subset='dept'))

It's actually sufficient to sort just by salary as the first value encountered for each department will be the employee with the highest salary. Note how the final DataFrame will be sorted by salary and not by department.

In [ ]:
(emp.sort_values('salary', ascending=False)
    .drop_duplicates(subset='dept'))

This short chain of steps combining `sort_values` with `drop_duplicates` is a generic and common pattern for finding the maximum or minimum of some column within groups formed by other columns. Below, we find the minimum salary for every unique combination of department, race, and sex.

In [ ]:
(emp.sort_values('salary')
    .drop_duplicates(subset=['dept', 'race', 'sex']).head())

## The `value_counts` method

The DataFrame `value_counts` method allows counting of unique values just like its counterpart Series method. Pass in the name of the column to could as the first argument.

In [ ]:
emp.value_counts('race')

Multiple column counts are possible by using a list. By default the groups are sorted in descending order. A Series containing a **multilevel** index is returned, which is a more advanced index that contains multiple labels for each value.

In [ ]:
emp.value_counts(['race', 'sex'])

Use parameter `normalize` to return the relative frequency and `ascending` to sort in reverse order.

In [ ]:
emp.value_counts(['dept', 'sex'], normalize=True, ascending=True).round(3).head() * 100

### Brief explanation of the multilevel index

pandas allows for the index (and columns) to have multiple levels. Each level is analogous to an extra column (or row)  but is technically still a label for the value. The Series above has two index levels and one sequence of values. Series always have one sequence of values but can have multiple levels. This multilevel index has names for each of the levels (dept and sex) that appear directly above them. A separate part of the book will be dedicated to multilevel indexes.

## Exercises

Use the employee dataset for the first few exercises.

### Exercise 1

<span  style="color:green; font-size:16px">Sort department, race, sex ascending along with salary descending.</span>

### Exercise 2

<span  style="color:green; font-size:16px">How many unique combinations of department and title exist?</span>

### Exercise 3

<span  style="color:green; font-size:16px">Since only Series methods have a `unique` method, can you think of a creative way of getting the same result as exercise 2 with the `unique` method?</span>

### Exercise 4

<span style="color:green; font-size:16px">Find the frequency of occurrence of all race and sex combinations using the trick from exercise 3. For instance, you would return an object that contains the number of 'Hispanic Males', 'Black Females', etc...</span>

### Use the college dataset for the remaining exercises

Execute the following cell to read in the college dataset which sets the institution name (`instnm`) as the index.

In [ ]:
college = pd.read_csv('../data/college.csv', index_col='instnm')
college.head(3)

### Exercise 5

<span style="color:green; font-size:16px">Select the columns `stabbr`, `satvrmid`, `satmtmid` and `ugds` columns for the state of Texas ('TX') that have an undergraduate student population of more than 20,000. Drop any rows with missing values and assign the result to the variable name `college_tx`. </span>

### Exercise 6

<span style="color:green; font-size:16px">Rank each column from the `college_tx` DataFrame from greatest to least.</span>

### Exercise 7

<span style="color:green; font-size:16px">Using the full college dataset, find the largest school by population for each state. Return only the `stabbr` and `ugds` columns sorting by `ugds`.</span>

### Exercise 8

<span  style="color:green; font-size:16px">Several of the columns from the college dataset contain binary data (are either 0 or 1). Can you identify the names of these columns?</span>